In [ ]:
#| default_exp handlers.maris_dump

# MARIS dump
> Data pipeline (handler) to convert global MARIS db dump into `NetCDF` format

The input data is a dump from already imported MARIS datasets.

**Dev. board**: https://trello.com/b/IszgV1bj/marisco

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from tqdm import tqdm
from pathlib import Path
import fastcore.all as fc
import pandas as pd

from marisco.callbacks import (Callback, Transformer, SanitizeLonLatCB, EncodeTimeCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB,
                              DepthRangeCB, TimeRangeCB,
                              ZoteroCB, KeyValuePairCB)
from marisco.configs import lut_path, cdl_cfg, cfg, nc_tpl_path, Enums
from marisco.serializers import NetCDFEncoder

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# fname_in = Path().home() / 'pro/data/maris/MARIS_exportSample_20240312_tab2.txt'
fname_in = Path().home() / 'pro/data/maris/MARIS_exportSample_20240313.txt'
dir_dest = '../../_data/output/dump'

## Utils

In [ ]:
# | export
def load_dump(fname): 
    return pd.read_csv(fname, sep='\t', encoding='ISO-8859-1')   

def load_data(df:pd.DataFrame, # MARIS global dump 
                 ref_id:int, # Reference id of interest
                 ):
    "Load specific MARIS dataset through its ref_id"
    lut = {
        'Sediment': 'sediment',
        'Seawater': 'seawater',
        'Suspended matter': 'suspended-matter',
        'Biota': 'biota'}
    dfs = {}
    for name, grp in df[df.ref_id  == ref_id].groupby('samptype'): 
        dfs[lut[name]] = grp
    return dfs

def get_zotero_key(dfs):
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

def get_fname(dfs):
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
df = load_dump(fname_in)

In [ ]:
df.head()

,ref_id,displaytext,samptype,nuclide_id,latitude,longitude,begperiod,endperiod,sampdepth,totdepth,...,bodypar_id,sedtype_id,volume,salinity,temperatur,sampmet_id,prepmet_id,counmet_id,activity,zoterourl
0,182,"Urban et al., 2015",Biota,33,-35.140833,117.604444,2014-05-06 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.387,https://www.zotero.org/groups/2432820/maris/it...
1,182,"Urban et al., 2015",Biota,47,-35.140833,117.604444,2014-05-06 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,5,1.44,https://www.zotero.org/groups/2432820/maris/it...
2,182,"Urban et al., 2015",Biota,31,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.042,https://www.zotero.org/groups/2432820/maris/it...
3,182,"Urban et al., 2015",Biota,33,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.075,https://www.zotero.org/groups/2432820/maris/it...
4,182,"Urban et al., 2015",Biota,47,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,5,0.069,https://www.zotero.org/groups/2432820/maris/it...


In [ ]:
# 52, 191 (OSPAR), 100 (HELCOM), 717 (only seawater)
ref_id = 100

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
dfs.keys()

dict_keys(['biota', 'seawater', 'sediment'])

## Data transformation pipeline

### Normalize nuclide names

In [ ]:
#| export
def get_varnames_lut():
    fname = lut_path() / 'dbo_nuclide.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['nuclide_id', 'nc_name'])
    return df_nuclide.set_index('nuclide_id').to_dict()['nc_name']

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=get_varnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['nuclide_id'] = tfm.dfs[k]['nuclide_id'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[RemapRdnNameCB()])

print(tfm()['sediment']['nuclide_id'].unique())

['ru106' 'sb125' 'cs134' 'cs137' 'k40' 'co60' 'ag110m' 'ra226' 'th232'
 'pb212' 'pb214' 'pu238' 'am241' 'pu239_240_tot' 'zr95' 'mn54' 'ac228'
 'u235' 'tl208' 'be7' 'bi214' 'ra223' 'ru103' 'sr90' 'eu155' 'ba140'
 'co58' 'ra224' 'po210' 'ra228' 'th228' 'ce144' 'cs134_137_tot' 'pb210'
 'pu239' 'cd109' 'bi212' 'pu238_240_tot' 'nb95' 'ir192' 'sb124' 'zn65'
 'th234' 'pu241']


### Rename columns

In [ ]:
dfs['sediment'].columns

Index(['ref_id', 'displaytext', 'samptype', 'nuclide_id', 'latitude',
       'longitude', 'begperiod', 'endperiod', 'sampdepth', 'totdepth',
       'uncertaint', 'unit_id', 'detection', 'area_id', 'species_id',
       'biogroup_id', 'bodypar_id', 'sedtype_id', 'volume', 'salinity',
       'temperatur', 'sampmet_id', 'prepmet_id', 'counmet_id', 'activity',
       'zoterourl'],
      dtype='object')

In [ ]:
#| export
# To be added: endperiod, totdepth, lab
def renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'latitude': vars['defaults']['lat']['name'],
        'longitude': vars['defaults']['lon']['name'],
        'begperiod': vars['defaults']['time']['name'],
        'sampdepth': vars['defaults']['depth']['name'],
        'uncertaint': vars['suffixes']['uncertainty']['name'],
        'unit_id': vars['suffixes']['unit']['name'],
        'detection': vars['suffixes']['detection_limit']['name'],
        'area_id': vars['defaults']['area']['name'], 
        'species_id': vars['bio']['species']['name'],
        'biogroup_id': vars['bio']['bio_group']['name'],
        'bodypar_id': vars['bio']['body_part']['name'],
        'sedtype_id': vars['sed']['sed_type']['name'],
        'volume': vars['suffixes']['volume']['name'],
        'salinity': vars['suffixes']['salinity']['name'],
        'temperatur': vars['suffixes']['temperature']['name'],
        'sampmet_id': vars['suffixes']['sampling_method']['name'],
        'prepmet_id': vars['suffixes']['preparation_method']['name'],
        'counmet_id': vars['suffixes']['counting_method']['name'],
        'activity': 'value',
        'nuclide_id': 'nuclide'
    }

In [ ]:
#| export
class RenameColumnCB(Callback):
    "Renaming variables to MARIS standard names."
    def __init__(self, renaming_rules=renaming_rules): fc.store_attr()
    def __call__(self, tfm):
        lut = renaming_rules()
        coi = lut.keys()
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, coi]
            # Rename cols
            tfm.dfs[k].rename(columns=lut, inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB()
    ])

print(tfm()['sediment'])

               lat        lon                 time depth       unc unit dl  \
549778   54.838333        9.9  1989-06-14 00:00:00  -1.0      3.99    4  =   
549779   54.838333        9.9  1989-06-14 00:00:00  -1.0       NaN    2  =   
549780   54.838333        9.9  1989-06-14 00:00:00  -1.0     1.674    4  =   
549781   54.838333        9.9  1989-06-14 00:00:00  -1.0       NaN    2  =   
549782   54.838333        9.9  1989-06-14 00:00:00  -1.0     1.829    4  =   
...            ...        ...                  ...   ...       ...  ... ..   
1532415  57.619722  23.621389  2005-12-02 00:00:00  -1.0   86.2836    4  =   
1532416  57.619722  23.621389  2005-12-02 00:00:00  -1.0       NaN    2  =   
1532417  57.619722  23.621389  2005-12-02 00:00:00  -1.0  24.45552    4  =   
1532418  57.619722  23.621389  2005-12-02 00:00:00  -1.0       NaN    2  =   
1532419  57.619722  23.621389  2005-12-02 00:00:00  -1.0  123.2568    4  =   

         area species bio_group body_part sed_type  vol  sal te

### Drop NaN only columns

In [ ]:
#| export
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0):
        fc.store_attr()
        
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB()
    ])

print(tfm()['sediment'])

             lat        lon                 time     unc  unit dl  area  \
14936 -62.166667 -94.276111  2003-01-31 00:00:00    5.00     4  =  1907   
14937 -62.166667 -94.276111  2003-01-31 00:00:00  222.00     4  =  1907   
14938 -62.166667 -94.276111  2003-01-31 00:00:00    0.10     4  =  1907   
14939 -62.166667 -94.276111  2003-01-31 00:00:00    0.07     4  =  1907   
14940 -62.166667 -94.276111  2003-01-31 00:00:00   40.00     4  =  1907   
15080 -62.169444 -58.601944  2003-01-31 00:00:00    0.71     4  =  1907   
15081 -62.169444 -58.601944  2003-01-31 00:00:00    0.79     4  =  1907   
15082 -62.172500 -58.546667  2003-01-31 00:00:00    0.50     4  =  1907   
15083 -62.172500 -58.546667  2003-01-31 00:00:00    0.43     4  =  1907   
15084 -62.173889 -58.443889  2003-01-31 00:00:00    0.41     4  =  1907   
15085 -62.173889 -58.443889  2003-01-31 00:00:00    0.47     4  =  1907   
15086 -62.190556 -58.610556  2003-01-31 00:00:00    0.63     4  =  1907   
15087 -62.190556 -58.6105

### Sanitize detection limit values

In [ ]:
#| export
def get_dl_lut():
    fname = lut_path() / 'dbo_detectlimit.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['name', 'id'])
    return df_nuclide.set_index('name').to_dict()['id']

In [ ]:
get_dl_lut()

{'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| export
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=get_dl_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['dl'] = tfm.dfs[k]['dl'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

print(tfm()['sediment'])

               lat        lon                 time depth       unc unit  dl  \
549778   54.838333        9.9  1989-06-14 00:00:00  -1.0      3.99    4   1   
549779   54.838333        9.9  1989-06-14 00:00:00  -1.0       NaN    2   1   
549780   54.838333        9.9  1989-06-14 00:00:00  -1.0     1.674    4   1   
549781   54.838333        9.9  1989-06-14 00:00:00  -1.0       NaN    2   1   
549782   54.838333        9.9  1989-06-14 00:00:00  -1.0     1.829    4   1   
...            ...        ...                  ...   ...       ...  ...  ..   
1532415  57.619722  23.621389  2005-12-02 00:00:00  -1.0   86.2836    4   1   
1532416  57.619722  23.621389  2005-12-02 00:00:00  -1.0       NaN    2   1   
1532417  57.619722  23.621389  2005-12-02 00:00:00  -1.0  24.45552    4   1   
1532418  57.619722  23.621389  2005-12-02 00:00:00  -1.0       NaN    2   1   
1532419  57.619722  23.621389  2005-12-02 00:00:00  -1.0  123.2568    4   1   

         area sed_type sampmet prepmet    value nuc

### Parse time

In [ ]:
#| export
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['time'] = pd.to_datetime(tfm.dfs[k].time, format='ISO8601')

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB()
    ])

print(tfm()['sediment'])

               lat        lon       time depth       unc unit  dl  area  \
549778   54.838333        9.9 1989-06-14  -1.0      3.99    4   1  2374   
549779   54.838333        9.9 1989-06-14  -1.0       NaN    2   1  2374   
549780   54.838333        9.9 1989-06-14  -1.0     1.674    4   1  2374   
549781   54.838333        9.9 1989-06-14  -1.0       NaN    2   1  2374   
549782   54.838333        9.9 1989-06-14  -1.0     1.829    4   1  2374   
...            ...        ...        ...   ...       ...  ...  ..   ...   
1532415  57.619722  23.621389 2005-12-02  -1.0   86.2836    4   1  2409   
1532416  57.619722  23.621389 2005-12-02  -1.0       NaN    2   1  2409   
1532417  57.619722  23.621389 2005-12-02  -1.0  24.45552    4   1  2409   
1532418  57.619722  23.621389 2005-12-02  -1.0       NaN    2   1  2409   
1532419  57.619722  23.621389 2005-12-02  -1.0  123.2568    4   1  2409   

        sed_type sampmet prepmet    value nuclide  
549778        59       0       0     26.6   ru1

### Reshape: long to wide

In [ ]:
set(['a', 'b'] + ['c', 'b'])

{'a', 'b', 'c'}

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns='nuclide', values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}_{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        
        df.reset_index(names='sample', inplace=True)
        
        idx = list(set(df.columns) - set([self.columns] + derived_coi + self.values))
        # print('idx: ', idx)
        # print('derived_coi: ', derived_coi)
        return df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=0,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)
            # tfm.dfs[k].index.name = 'sample'
            tfm.dfs[k].set_index('sample', inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide()
    ])

print(tfm()['sediment'])

         sed_type       time  depth        lon  area        lat  ac228_dl  \
sample                                                                      
630057          0 1984-06-03   -1.0  14.418333  2401  54.333333         0   
630058          0 1984-06-03   -1.0  14.418333  2401  54.333333         0   
630059          0 1984-06-03   -1.0  14.418333  2401  54.333333         0   
630060          0 1984-06-03   -1.0  14.418333  2401  54.333333         0   
630061          0 1984-06-03   -1.0  14.418333  2401  54.333333         0   
...           ...        ...    ...        ...   ...        ...       ...   
1456914        62 1998-09-23   -1.0  12.250000  2401  54.583333         0   
1484005        62 1998-09-23   -1.0  12.250000  2401  54.583333         0   
1484006        62 1998-09-23   -1.0  12.250000  2401  54.583333         0   
1484007        62 1998-09-23   -1.0  12.250000  2401  54.583333         0   
1484008        62 1998-09-23   -1.0  12.250000  2401  54.583333         0   

### Encode time (seconds since ...)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg())
    ])

print(tfm()['sediment'])

         sed_type       time  depth        lon  area        lat  ac228_dl  \
sample                                                                      
630057          0  455068800   -1.0  14.418333  2401  54.333333         0   
630058          0  455068800   -1.0  14.418333  2401  54.333333         0   
630059          0  455068800   -1.0  14.418333  2401  54.333333         0   
630060          0  455068800   -1.0  14.418333  2401  54.333333         0   
630061          0  455068800   -1.0  14.418333  2401  54.333333         0   
...           ...        ...    ...        ...   ...        ...       ...   
1456914        62  906508800   -1.0  12.250000  2401  54.583333         0   
1484005        62  906508800   -1.0  12.250000  2401  54.583333         0   
1484006        62  906508800   -1.0  12.250000  2401  54.583333         0   
1484007        62  906508800   -1.0  12.250000  2401  54.583333         0   
1484008        62  906508800   -1.0  12.250000  2401  54.583333         0   

### Sanitize coordinates

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

# print(tfm()['sediment'])
df_debug = tfm()['sediment']

## Encode to NetCDF

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()

In [ ]:
tfm.logs

['Remap to MARIS radionuclide names.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx',
 'Drop row when both longitude & latitude equal 0.']

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| export
def get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '-125.49983',
 'geospatial_lat_max': '50.002',
 'geospatial_lon_min': '-145.00116',
 'geospatial_lon_max': '48.574',
 'geospatial_bounds': 'POLYGON ((-145.00116 48.574, -125.49983 48.574, -125.49983 50.002, -145.00116 50.002, -145.00116 48.574))',
 'geospatial_vertical_max': '-2.1',
 'geospatial_vertical_min': '-1001.6',
 'time_coverage_start': '2011-06-05T00:00:00',
 'time_coverage_end': '2019-08-25T00:00:00',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}]',
 'keywords': 'oceanograph

In [ ]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    df = load_dump(fname_in)
    ref_ids = kwargs.get('ref_ids', df.ref_id.unique())
    print('Encoding ...')
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = load_data(df, ref_id)
        print(get_fname(dfs))
        tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(cfg()),
            SanitizeLonLatCB(verbose=True)
            ])
       
        tfm()
        encoder = NetCDFEncoder(tfm.dfs, 
                                src_fname=nc_tpl_path,
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False),
                                enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                                )
        encoder.encode()

### Single dataset

In [ ]:
tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(cfg()),
            SanitizeLonLatCB(verbose=True)
            ])
tfm()

{'biota':                value        time  depth  bio_group        lon  area  \
 sample                                                                
 662144      0.000000  1537401600   -1.0          4  14.500000  2401   
 1496956     0.000000  1537401600   -1.0          4  14.500000  2401   
 648566      0.000002   786153600   49.5          4  20.560000  2401   
 1448546     0.000002   786153600   49.5          4  20.560000  2401   
 648565      0.000003   786153600   49.5          4  20.560000  2401   
 ...              ...         ...    ...        ...        ...   ...   
 1507441  2100.000000   526521600    0.0         11  18.366667  2402   
 637491   2210.000000   565920000   14.5         11  10.510000  2401   
 1454028  2210.000000   565920000   14.5         11  10.510000  2401   
 711921   2850.000000   803260800    0.0         11  23.500000  2409   
 1532087  2850.000000   803260800    0.0         11  23.500000  2409   
 
          body_part        lat  species  ac228_counme

In [ ]:
ref_id = 12
encode(fname_in, dir_dest, nc_tpl_path(), verbose=True, ref_ids=[ref_id])

Encoding ...


  0%|          | 0/1 [00:00<?, ?it/s]

12-Higgo-et-al-1980.nc
The "biota" group contains 16 data points whose (lon, lat) = (0, 0)
Group: biota, Variable: sample
Group: biota, Variable: lon
Group: biota, Variable: lat
Group: biota, Variable: time
Group: biota, Variable: area
Group: biota, Variable: bio_group
Group: biota, Variable: species
Group: biota, Variable: body_part
Group: biota, Variable: po210_unc
Group: biota, Variable: po210_dl
Group: biota, Variable: po210_counmet
Group: biota, Variable: po210_sampmet
Group: biota, Variable: po210_unit
Group: biota, Variable: th232_unc
Group: biota, Variable: th232_dl
Group: biota, Variable: th232_counmet
Group: biota, Variable: th232_sampmet
Group: biota, Variable: th232_unit
Group: biota, Variable: u238_unc
Group: biota, Variable: u238_dl
Group: biota, Variable: u238_counmet
Group: biota, Variable: u238_sampmet
Group: biota, Variable: u238_unit
Group: biota, Variable: pu239_240_tot_unc
Group: biota, Variable: pu239_240_tot_dl
Group: biota, Variable: pu239_240_tot_counmet
Group:

### All datasets

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False)

Encoding ...


  0%|          | 0/526 [00:00<?, ?it/s]

182-Urban-et-al-2015.nc


  0%|          | 1/526 [00:02<21:30,  2.46s/it]

183-Bokor-et-al-2016.nc


...

270-Garcia-Orellana-et-al-2016.nc


 88%|████████▊ | 461/526 [39:44<02:07,  1.96s/it]

294-Bode-et-al-2015.nc


697-Long-et-al-1998.nc


TypeError: ufunc 'points' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''